# ECBM E4040 - Assignment 2 - Task 3: Convolutional Neural Network (CNN)

In this task, you are going to first practice the forward/backward propagation of the convolutional operations with Numpy. After that, we will introduce TensorFlow with which you'll create your CNN model for an image classification task. 

## CNNs:
This is one of the good posts describing CNNs:

[https://adeshpande3.github.io/adeshpande3.github.io/A-Beginner%27s-Guide-To-Understanding-Convolutional-Neural-Networks/](https://adeshpande3.github.io/adeshpande3.github.io/A-Beginner%27s-Guide-To-Understanding-Convolutional-Neural-Networks/)

Convolutional neural networks (CNNs) are a type of neural networks which is highly effective for image processing. 

Remember when we build an MLP model, each input is multiplied by its own weights. When the input dimension or the first layer is too large, we will need a giant matrix to store our weights. This could easily become a problem in image processing since the dimension of a vectorized image could easily exceed 1000 (consider CIFAR-10 which has images of shape 32×32=1024, yet the resolution is so low). 

In CNN, the weights can be shared: the same filter (also known as 'weights' or 'kernel') moves over the input, and at each position an output value is calculated. This means the same weights are used by the entire input, therefore saving a lot of memory.

![Illustration of the CNN](./ecbm4040/notebook_images/task3_1.jpg)
Image source: [here](https://developer.apple.com/library/content/documentation/Performance/Conceptual/vImage/ConvolutionOperations/ConvolutionOperations.html)

__Convolution:__  In the picture above, the input is a 7-by-7 image, and the filter is shown as a blue 3-by-3 grid. The filter overlaps with the top-left corner of the input, and we perform an element-wise multiplication followed by a summation, then put the sum into the output matrix. The filter then moves several pixels right, covering a new input area so a new sum can be derived.

__Training:__ One thing to remember is that there would be a lot of filters for each layer in a CNN, and the goal of training is to find the best filters for your task. Each filter tries to capture one specific feature. Typically, in the first convolutional layer which directly looks at your input, the filters try to capture information about color and edges which we know as local features; in higher layers, due to the effect of max-pooling, the receptive-fields of filters becomes large so more global and complex features can be detected. 

__Architecture:__ For classification tasks, a CNN usually starts with convolution followed by max-pooling. After that, the feature maps will be flattened so that we could append fully connected layers. Common activation functions include ReLu, ELU in the convolution layers, and softmax in the fully connected layers (to calculate the classification scores).

---

### Terminology

* __Convolution__: element-wise multiplication followed by summation of your input and one of your filters in the CNN context.
* __Filter/kernel/weights__: a grid or a set of grids typically smaller than your input size that moves over the input space to generate output. Each filter captures one type of feature.
* __Feature/feature maps__: the output of a hidden layer. Think of it as another representation of your data. 
* __Pooling__: an downsampling operation that joins local information together, so the higher layers' receptive fields can be bigger. The most seen pooling operation is max-pooling, which outputs the maximum of all values inside the pool.
* __Flatten__: a junction between convolution layers and fully connected layers. Used to turn 2-D feature maps into 1-D. For tasks such as image segmentation where the output also needs to be 2-D, this won't be used.
* __Border mode__: usually refers to 'VALID' or 'SAME'. Under 'VALID' mode, only when the filter and the input fully overlap can a convolution be conducted; under 'SAME' mode, the output size is the same as the input size, and when the filter and the input don't fully overlap (happens at the edge/corner of input) we pad zeroes (or other designated numbers) and then do convolution.

In [128]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Import modules
from __future__ import print_function
import tensorflow as tf
import numpy as np
from ecbm4040.cifar_utils import load_data
from matplotlib import pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Part 1: Getting a sense of convolution

### conv2d feedforward

Implement a Numpy naive 2-D convolution feedforward function. We ask you to simply do the element-wise multiplication and summation. Also, don't need to worry about the efficiency of your function. Use loops as many as you like.

<span style="color:red">__TODO:__</span> Finish the function __conv2d_forward__ in __ecbm4040/layer_funcs.py__. After that, run the following cell blocks, which will give the output of your convolution function. Detailed instructions have been given in the comments of __layer_func.py__. __We need to judge your output to give you credits__.

In [120]:
from ecbm4040.layer_funcs import conv2d_forward

# Set test parameters.
x_shape = (2, 4, 4, 3)
w_shape = (3, 4, 4, 3)
x = np.linspace(-0.1, 0.5, num=np.prod(x_shape)).reshape(x_shape)
w = np.linspace(-0.2, 0.3, num=np.prod(w_shape)).reshape(w_shape)
b = np.linspace(-0.1, 0.2, num=3)
pad = 1
stride = 2
your_feedforward = conv2d_forward(x, w, b, pad, stride)
print(your_feedforward)

[[[[-0.04138903  0.11338093  0.2681509 ]
   [-0.09391829  0.14671108  0.38734045]]

  [[-0.31590063  0.18230696  0.68051454]
   [-0.41135959  0.1727074   0.75677438]]]


 [[[-0.79122782  0.7372926   2.26581303]
   [-0.92961649  0.68476334  2.29914317]]

  [[-1.40917703  0.46278101  2.33473905]
   [-1.5904954   0.36732205  2.32513949]]]]


### conv2d backpropogation (optional, bonus +10 points)

<p style="color:red">This function is optional, but a bonus 10 points will be given if you solve it correctly.</p>

Implement a Numpy naive 2-D convolution backpropagation function. Again, don't worry about the efficienty.

<span style="color:red">__TODO:__</span> Finish the function __conv2d_backward__ in __ecbm4040/layer_funcs.py__. After that, run the following cell blocks, which will give the output of your backpropagation. Detailed instructions have been given in the comments of __layer_func.py__. __We need to judge your output to give you credits__.

In [123]:
from ecbm4040.layer_funcs import conv2d_backward
# Set test parameters. Please don't change it.
np.random.seed(123)
d_top = np.random.normal(size=your_feedforward.shape)
your_dw, your_db = conv2d_backward(d_top, x, w, b, pad, stride)
print(your_dw)
print('*'*50)
print(your_db)

[[[[  2.24995483e-01   2.26276827e-01   2.27558171e-01]
   [  1.86493037e-01   1.81646025e-01   1.76799014e-01]
   [  1.71952002e-01   1.67104991e-01   1.62257979e-01]
   [ -7.91166133e-02  -8.52449690e-02  -9.13733248e-02]]

  [[  4.50218811e-01   4.47007654e-01   4.43796497e-01]
   [  5.19462603e-01   5.12093590e-01   5.04724578e-01]
   [  4.97355565e-01   4.89986552e-01   4.82617540e-01]
   [  5.39301272e-02   4.97722713e-02   4.56144154e-02]]

  [[  4.11684929e-01   4.08473773e-01   4.05262616e-01]
   [  4.31034451e-01   4.23665438e-01   4.16296425e-01]
   [  4.08927413e-01   4.01558400e-01   3.94189387e-01]
   [  4.03585658e-03  -1.21999302e-04  -4.27985519e-03]]

  [[  1.02027171e-01   9.75346697e-02   9.30421686e-02]
   [  3.30605675e-01   3.28083674e-01   3.25561672e-01]
   [  3.23039671e-01   3.20517670e-01   3.17995669e-01]
   [  2.53879006e-01   2.55849506e-01   2.57820006e-01]]]


 [[[ -9.77873644e-02  -9.66554377e-02  -9.55235110e-02]
   [  2.20281720e-01   2.26552583e-01 

## Part 2: TensorFlow CNN

In this part we will construct the CNN in TensorFlow. To be more specific, we are going to implement a CNN similar to the LeNet structure.

Tensorflow offers many useful resources and functions which help developers build the net in a high-level fashion, such as functions in the `layer` module. However, we will build the network by ourself for this homework for better understanding. By utilizing functions in `tf.nn` that exist for Neural Network structuring and training, we can build out our own layers and network modules rather quickly.

Also, we will introduce a visualization tool called Tensorboard. You can use TensorBoard to visualize your TensorFlow graph, plot quantitative metrics about the execution of your graph, and show additional data that pass through it.

Resources and References: <br>
* [TensorBoard: Visualizing Learning](https://www.tensorflow.org/get_started/summaries_and_tensorboard)<br>
* [Convolutional Neural Networks (LeNet) - DeepLearning 0.1 documentation](http://deeplearning.net/tutorial/lenet.html)<br>
* [LeNet-5, convolutional neural networks](http://yann.lecun.com/exdb/lenet/)

### Quick guide for Tensorboard

Tensorboard is a powerful tool provided by TensorFlow. It allows developers to check their graph and trend of parameters. This guide will give you a basic under standing on how to set up Tensorboard graph in your code, start tensorboard on your local machine/GCP instance and how to access the interface.

For complete instructions, check the official guide on Tensorflow web site [here](https://www.tensorflow.org/get_started/summaries_and_tensorboard).

### How to start tensorboard

#### Local

To start your Tensorboard on your local machine, you need to specify a log directory for the service to fetch the graph. For example, in your command line, type:

```shell
$ tensorboard --logdir="~/log"
```

Then, Tensorboard will start running. By default, it will be running on port 6006:

``` shell
TensorBoard 0.1.6 at http://localhost:6006 (Press CTRL+C to quit)
```

Make sure Tensorboard is running, you can visit http://localhost:6006 In your browser and you should be able to see the main page of Tensorboard. If the page is shown as below, it means Tensorboard is running correctly. The report is due to lack of event file, but we can just leave it there for now.

![Tensorboard_1](./ecbm4040/notebook_images/task3_2_1.png)

#### GCP

To set up the Tensorboard on GCP is the same as above. However, we're not able to check the Tensorboard UI directly through our browser. In order to visit the page through our local browser, we should link the port of our local machine to the port on GCP. It is similar to what we did previously for Jupyter Notebook.

In the command line on your local machine, type:

```shell
$ gcloud compute ssh --ssh-flag="-L 9999:localhost:9999 -L 9998:localhost:6006" "ecbm4040@YOUR_INSTANCE"
```

 This will bind your port of your local machine to the port on GCP instance. In this case, your local port 9999 is binded with 9999 on GCP, while local port 9998 is binded with 6006 on GCP. You can change whatever port you like as long as it does not confilct with your local services.

After connecting to GCP using the command, you will be able to see the result page.



### Export Tensorboard events into log directory

To generate data files for Tensorboard, we should use class `tf.summary.FileWriter`. This class will save your network graph sturcuture and all the variable summary. 

For example, in `cnn_sample.py `, the file writer will save the graph and the summary into a directory based on the current timestamp. Here is the code snippet:

```python
cur_model_name = 'lenet_{}'.format(int(time.time()))
# ...

# set up summary writer for tensorboard
merge = tf.summary.merge_all()	# merge all the summary for variables for execution
writer = tf.summary.FileWriter("log/{}".format(cur_model_name), sess.graph)
```

The following code will save all the parameter summary and marked with iteration_total. These data will be displayed in the Tensorboard latter on.

```python
# ... previous code ...
# ...
				if iter_total % 100 == 0:
                    # do validation
                    valid_eve, merge_result = sess.run([eve, merge], feed_dict={xs: X_val, ys: y_val})
                    valid_acc = 100 - valid_eve * 100 / y_val.shape[0]
                    if verbose:
                        print('{}/{} loss: {} validation accuracy : {}%'.format(
                            batch_size * (itr + 1),
                            X_train.shape[0],
                            cur_loss,
                            valid_acc))

                    # save the merge result summary
                    writer.add_summary(merge_result, iter_total)
```



### Check the graph and summary in Tensorboard

After executing the program once, you should able to see the graph displayed in the tensorboard. You can zoom in or zoom out or click into the layer block to check all the variables and tensor operations.

![Tensorboard_2](./ecbm4040/notebook_images/task3_2_2.png)

Also, you may able to check the trend of the variables and the distribution of those in Scalar, Distributions and Histograms. You may explore the tensorboard by yourself and take advantage to it for debuging the nerwork structure.

<span style="color:red">__TODO:__</span> You will try to achieve your own CNN model that has similar structure to LeNet, show the model graph in tensorboard, and get a model with **65%** or higher accuracy using the data we provide you.

An example code is included in __ecbm4040/neuralnets/cnn_sample.py__. This sample is used as a guide line for how to build a Neural Net model in Tensorflow. Feel free to utilize or change the code we give you.

### Load Data
<p style="color:red">The following cell load the data for you. You don't need to change them.</p>

In [124]:
# Load the raw CIFAR-10 data.
X_train, y_train = load_data(mode='train')

# Data organizations:
# Train data: 49000 samples from original train set: 1~49000
# Validation data: 1000 samples from original train set: 49000~50000
num_training = 49000
num_validation = 1000

X_val = X_train[-num_validation:, :]
y_val = y_train[-num_validation:]

X_train = X_train[:num_training, :]
y_train = y_train[:num_training]

# Preprocessing: subtract the mean value across every dimension for training data, and reshape it to be RGB size
mean_image = np.mean(X_train, axis=0)
X_train = X_train.astype(np.float32) - mean_image.astype(np.float32)
X_val = X_val.astype(np.float32) - mean_image

X_train = X_train.reshape([-1,32,32,3])/255
X_val = X_val.reshape([-1,32,32,3])/255

print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)

./data/cifar-10-python.tar.gz already exists. Begin extracting...
Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000,)
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)


### CNN model example

In [129]:
from ecbm4040.neuralnets.cnn_sample import training
tf.reset_default_graph()
training(X_train, y_train, X_val, y_val, 
         conv_featmap=[6],
         fc_units=[84],
         conv_kernel_size=[5],
         pooling_size=[2],
         l2_norm=0.01,
         seed=235,
         learning_rate=1e-2,
         epoch=20,
         batch_size=245,
         verbose=False,
         pre_trained_model=None)

Building my LeNet. Parameters: 
conv_featmap=[6]
fc_units=[84]
conv_kernel_size=[5]
pooling_size=[2]
l2_norm=0.01
seed=235
learning_rate=0.01
number of batches for training: 200
epoch 1 
Best validation accuracy! iteration:100 accuracy: 16.5%
Best validation accuracy! iteration:200 accuracy: 18.400000000000006%
epoch 2 
Best validation accuracy! iteration:300 accuracy: 18.5%
Best validation accuracy! iteration:400 accuracy: 20.0%
epoch 3 
Best validation accuracy! iteration:500 accuracy: 21.599999999999994%
Best validation accuracy! iteration:600 accuracy: 21.799999999999997%
epoch 4 
Best validation accuracy! iteration:700 accuracy: 22.5%
Best validation accuracy! iteration:800 accuracy: 23.700000000000003%
epoch 5 
Best validation accuracy! iteration:900 accuracy: 24.200000000000003%
Best validation accuracy! iteration:1000 accuracy: 25.299999999999997%
epoch 6 
Best validation accuracy! iteration:1100 accuracy: 25.700000000000003%
Best validation accuracy! iteration:1200 accuracy: 2

### Show the model structure graph

In [130]:
# show the graph
from ecbm4040.neuralnets.cnn_jupyter_tensorboard import show_graph 
tf.reset_default_graph()
with tf.Session() as sess:
    saver = tf.train.import_meta_graph('model/lenet_1508771191.meta')
    graph = tf.get_default_graph()
    show_graph(graph)

### Custom CNN model

<span style="color:red">__TODO:__</span> 
1. Complete the your own CNN in __ecbm4040/neuralnets/cnn.py__ with at least **65%** accuracy.
2. Print out the training process and the best validation accuracy, save the `.meta` model in __model/__ folder.
3. Attatch a screen shot of your tensorboard graph in the markdown cell below. Double click the cell and replace the example image with your own image. Here is a [Markdown Cheetsheet](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#images) that may also help.

__Hint__: 
1. You can copy and edit the code from `cnn_sample.py`
2. The techniques in task-1 and task-2 will help. Check the corresponding functions in Tensorflow.

In [148]:
from ecbm4040.neuralnets.cnn import my_training
tf.reset_default_graph()
my_training(X_train, y_train, X_val, y_val, 
         conv_featmap=[32,32],
         fc_units=[400],
         conv_kernel_size=[4,4],
         pooling_size=[2,2],
         l2_norm=0.01,
         seed=235,
         learning_rate=1e-2,
         epoch=30,
         batch_size=245,
         verbose=False,
         pre_trained_model=None)

Building my LeNet. Parameters: 
conv_featmap=[32, 32]
fc_units=[400]
conv_kernel_size=[4, 4]
pooling_size=[2, 2]
l2_norm=0.01
seed=235
learning_rate=0.01
number of batches for training: 200
epoch 1 
Best validation accuracy! iteration:100 accuracy: 43.3%
Best validation accuracy! iteration:200 accuracy: 49.6%
epoch 2 
Best validation accuracy! iteration:300 accuracy: 51.9%
Best validation accuracy! iteration:400 accuracy: 54.6%
epoch 3 
Best validation accuracy! iteration:500 accuracy: 55.1%
Best validation accuracy! iteration:600 accuracy: 55.7%
epoch 4 
Best validation accuracy! iteration:700 accuracy: 57.0%
Best validation accuracy! iteration:800 accuracy: 59.2%
epoch 5 
Best validation accuracy! iteration:1000 accuracy: 60.0%
epoch 6 
Best validation accuracy! iteration:1100 accuracy: 61.2%
Best validation accuracy! iteration:1200 accuracy: 61.9%
epoch 7 
epoch 8 
Best validation accuracy! iteration:1600 accuracy: 62.7%
epoch 9 
Best validation accuracy! iteration:1700 accuracy: 62

<span style="color:red">__TODO:__</span> replace the example image with your own tensorboard graph screenshot.
![Tensorboard_2](./ecbm4040/notebook_images/my_lenet.png)

In [149]:
# show the graph
from ecbm4040.neuralnets.cnn_jupyter_tensorboard import show_graph 
tf.reset_default_graph()
with tf.Session() as sess:
    saver = tf.train.import_meta_graph('model/lenet_1508817660.meta')
    graph = tf.get_default_graph()
    show_graph(graph)